In [1]:
import numpy as np
import pandas as pd

In [2]:
def url(latitude, longitude, angle):
    '''
    Form the full URL to access data via the PVGIS API.
    
    Parameters:
        latitude: {float}
            Latitude of the location in decimal degrees.
        longitude: {float}
            Longitude of the location in decimal degrees.
        angle: {float}
            tilt angle for irradiation.
    Returns:
        {string}
    '''
    
    base_url = 'https://re.jrc.ec.europa.eu/api/v5_2/MRcalc' #The base url to access data via the PVGIS API.

    start_year = 2005 #First year for the desire outcome.
    end_year = 2020 #Final year for the desire outcome.

    parameters = {'lat': latitude,
                  'lon': longitude,
                  'raddatabase': 'PVGIS-ERA5', #Name of the radiation database to be used according to the chosen location.
                  'startyear': start_year,
                  'endyear': end_year,
                  'outputformat': 'csv', #Type of output: comma separated values (csv).
                  'angle': angle,
                  'selectrad': 1 #Output irradiation on plane of selected inclination.
                 }

    query_string = '&'.join([f'{key}={parameters[key]}' for key in parameters]) #Forming the query string.

    full_url = base_url + '?' + query_string #The full url to access data via the PVGIS API.

    return full_url

In [103]:
def optimum_solar_panel_tilt_angle(latitude, longitude, months = "Year"):
    '''
    Get the optimun tilt angle of solar paneles for maximun irradiation in a desired location in specific months.
    
    Parameters:
        latitude: {float}
            Latitude of the location in decimal degrees.
        longitude: {float}
            Longitude of the location in decimal degrees.
        months: {string, list}
            desired months to evaluate solar irradiation, default "Year". Other supported periods are:
                * "S1" : Semester 1.
                * "S2" : Semester 2.
                * "Q1" : Quarter 1.
                * "Q2" : Quarter 2.
                * "Q3" : Quarter 3.
                * "Q4" : Quarter 4.
            Lists must contain among the next string values:
                <"Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec">
    Returns:
        {int}
    '''
    if months == "Year":
        months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    elif months == "S1":
        months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]
    elif months == "S2":
        months = ["Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    elif months == "Q1":
        months = ["Jan", "Feb", "Mar"]
    elif months == "Q2":
        months = ["Apr", "May", "Jun"]
    elif months == "Q3":
        months = ["Jul", "Aug", "Sep"]
    elif months == "Q4":
        months = ["Oct", "Nov", "Dec"]
    
    angles = np.arange(0, 91) #Analyze angles from 0° to 90° with a step of 1°.
    
    total_irradiation_at_angles ={}
    
    for angle in angles:
        df = pd.read_csv(url(latitude, longitude, angle), header = 3, skipfooter = 4, sep = '\t\t', engine = 'python')
        df.rename(columns = {"H(i)_m": "montly_irradiation"}, inplace = True)
        df = df[df["month"].isin(months)]
        montly_mean_irradiation = df.groupby('month')["montly_irradiation"].mean()
        total_irradiation = montly_mean_irradiation.sum()
        total_irradiation_at_angles[angle] = total_irradiation
    
    return pd.Series(total_irradiation_at_angles).idxmax()

In [104]:
optimum_solar_panel_tilt_angle(latitude = -6.914, longitude = -79.450)

8